<a href="https://colab.research.google.com/github/rameshveer/School_of_AI-EVA_5/blob/master/S9/EVA5_S9_C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

S9 Assignment:

1. Move your last code's transformations to Albumentations. Apply ToTensor, HorizontalFlip, Normalize (at min) + More (for additional points)
2. Please make sure that your test_transforms are simple and only using ToTensor and Normalize
3. Implement GradCam function as a module. 
4. Your final code (notebook file) must use imported functions to implement transformations and GradCam functionality
5. Target Accuracy is 87%

In [ ]:
%matplotlib inline

Install albumentation code

In [ ]:
#!pip install albumentations==0.4.5

### Import Created libraries (Specific to this excercise)

In [ ]:
rm -rf convlib

In [ ]:
!git clone https://github.com/rameshveer/convlib.git

Cloning into 'convlib'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 153 (delta 4), reused 0 (delta 0), pack-reused 138
Receiving objects: 100% (153/153), 95.84 KiB | 13.69 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [ ]:
!pip install -e /content/convlib

Obtaining file:///content/convlib
  Found existing installation: convlib 1.0.0
    Can't uninstall 'convlib'. No files were found to uninstall.
  Running setup.py develop for convlib


### Importing Torch libraries


In [ ]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary

### Import created libraries 

In [ ]:
import convlib.models.resnet as model
import convlib.utils.cuda as cu
import convlib.utils.dataloader as dataloader
import convlib.utils.trainer as trainer
import convlib.utils.evaluate as evaluate
from convlib.data.dataset import CIFAR10


In [ ]:
#Setup params

class Args:
    # Training
    # ========
    random_seed = 1
    epochs = 20
    learning_rate = 0.01
    momentum = 0.9
    lr_step_size = 25
    lr_gamma = 0.1
    batch_size = 64
    num_workers=4
    sample_count = 25

    # Augmentation
    # ============
    horizontal_flip_prob = 0.2
    vertical_flip_prob = 0.0
    gaussian_blur_prob = 0.0
    rotate_degree = 20
    cutout = 0.3

    # Means & std:
    means = (0.5, 0.5, 0.5)
    stdevs = (0.5, 0.5, 0.5)


### Setup Device

In [ ]:
cuda, device = cu.init_cuda(Args.random_seed)
cuda

CUDA Available? True
Device: cuda


True

Apply Albumentation params in Transforms

In [ ]:
import albumentations as A

dataset = CIFAR10(
    train_batch_size=Args.batch_size,
    val_batch_size=Args.batch_size,
    cuda=cuda,
    num_workers=Args.num_workers,
    horizontal_flip_prob=Args.horizontal_flip_prob,
    vertical_flip_prob=Args.vertical_flip_prob,
    gaussian_blur_prob=Args.gaussian_blur_prob,
    rotate_degree=Args.rotate_degree,
    cutout=Args.cutout
)

Extracting /content/convlib/data/cifar10/cifar-10-python.tar.gz to /content/convlib/data/cifar10
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Create train data loader
train_loader = dataset.loader(train=True)

# Create val data loader
val_loader = dataset.loader(train=False)

In [ ]:

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Summary of model

In [ ]:
from torchsummary import summary

print(device)
model = model.ResNet18().to(device)
summary(model,input_size = (3,32,32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 32, 32]          73,728
      BatchNorm2d-14          [-1,

Train the model

In [ ]:
#Loss function and Optimizer - Let's use a Classification Cross-Entropy loss and SGD with momentum.

criterion = nn.CrossEntropyLoss()

from torch.optim.lr_scheduler import StepLR
import torch.optim as optim

losses = []
accuracies = []
correct_samples = []
incorrect_samples = []

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.3)
last_epoch = False

for epoch in range(1, Args.epochs + 1):
    print(f'Epoch {epoch}:')
    if epoch == Args.epochs:
        last_epoch = True
    trainer.train(model, train_loader, device, optimizer, criterion)
    scheduler.step()
    trainer.val(
        model, val_loader, device, criterion, losses, accuracies,
        correct_samples, incorrect_samples, Args.sample_count, last_epoch=last_epoch
    )


  0%|          | 0/782 [00:00<?, ?it/s]

Epoch 1:


  0%|          | 0/782 [00:00<?, ?it/s]


Validation set: Average loss: 0.0141, Accuracy: 6864/10000 (68.64%)

Epoch 2:


  0%|          | 0/782 [00:00<?, ?it/s]


Validation set: Average loss: 0.0118, Accuracy: 7429/10000 (74.29%)

Epoch 3:


Loss=0.62 Batch_ID=519 Accuracy=76.50:  66%|██████▋   | 519/782 [00:45<00:22, 11.47it/s]

Evalutate 